# Prompt
## PromptTemplate

In [1]:
from langchain.prompts import PromptTemplate

# Simple prompt with placeholders
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

# Filling placeholders to create a prompt
filled_prompt = prompt_template.format(adjective="funny", content="robots")
print(filled_prompt)
filled_prompt = prompt_template.invoke({"adjective":"funny", "content":"robots"})
print(filled_prompt)
filled_prompt.to_messages()

/Users/daijunjie/miniconda3/envs/langchain-env/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Tell me a funny joke about robots.
text='Tell me a funny joke about robots.'


[HumanMessage(content='Tell me a funny joke about robots.', additional_kwargs={}, response_metadata={})]

## ChatPromptTemplate

In [2]:
from langchain.prompts  import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ])

formatted_messages = chat_template.format_messages(name="Alice", user_input="What is the meaning of life?")

for message in formatted_messages:
    print(message)
    # print(message.model_construct())
print(formatted_messages)



content='You are a helpful AI bot. Your name is Alice.' additional_kwargs={} response_metadata={}
content='Hello, how are you doing?' additional_kwargs={} response_metadata={}
content="I'm doing well, thanks!" additional_kwargs={} response_metadata={}
content='What is the meaning of life?' additional_kwargs={} response_metadata={}
[SystemMessage(content='You are a helpful AI bot. Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the meaning of life?', additional_kwargs={}, response_metadata={})]


# JsonOutputParser

In [ ]:
from demo1 import get_model
model = get_model()
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with `birthdate` and `birthplace` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | model | json_parser

result_list = list(json_chain.stream({"question": "When and where was Elon Musk born?"}))[-1]
print(result_list)

# chain = json_prompt | model
# output = chain.invoke("When and where was Elon Musk born?")
# print(output.content)


content='{\n  "birthdate": "June 28, 1971",\n  "birthplace": "Pretoria, South Africa"\n}' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'qwq-plus-latest'} id='run--951faa7d-1365-4c45-9de9-1909c32be5af-0'


# CommaSeparatedListParser

In [19]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from demo1 import get_model
model = get_model()

# Initialize output parser
output_parser = CommaSeparatedListOutputParser()
# Create format instructions
format_instructions = output_parser.get_format_instructions()

# Create a prompt to request a list
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    # input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

# Define a query to prompt the model
query = "English Premier League Teams"
# Generate the output
output = model.invoke(prompt.format(subject=query)).content
print(output)
parsed_result = output_parser.parse(output)
print(parsed_result)

Manchester United, Manchester City, Liverpool, Chelsea, Arsenal
['Manchester United', 'Manchester City', 'Liverpool', 'Chelsea', 'Arsenal']


## chain版本的CommaSeparatedListParser

In [21]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from demo1 import get_model
model = get_model()

# Initialize output parser
output_parser = CommaSeparatedListOutputParser()
# Create format instructions
format_instructions = output_parser.get_format_instructions()

# Create a prompt to request a list
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    # input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

# Define a query to prompt the model
query = "English Premier League Teams"
# build the chain
chain = prompt | model | output_parser
# Run the chain 不用手动执行parse操作
output = chain.invoke({"subject": query})
print('output: ', output)
print('type: ', type(output))

output:  ['Manchester United', 'Manchester City', 'Liverpool', 'Chelsea', 'Arsenal']
type:  <class 'list'>


# DatetimeOutputParser

In [15]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser

from langchain.chains import LLMChain
from demo1 import get_model

model = get_model()
# Initialize the DatetimeOutputParser
output_parser = DatetimeOutputParser()

# Create a prompt with format instructions
template = """
Answer the user's question:
{question}
{format_instructions}
"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = LLMChain(llm=model, prompt=prompt)

query = "when did Neil Armstrong land on the moon in terms of GMT?"

# Run the chain
output = chain.run(query)
print('raw output:', output, '\n')
# Parse the output using the datetime parser
parsed_result = output_parser.parse(output)

print(parsed_result)

raw output: 1969-07-20T20:17:40.000000Z 

1969-07-20 20:17:40
